In [ ]:
# This cell imports the standard pandas library.
import pandas as pd

# SQL Tutorial

What is SQL:
- SQL is a language for managing Relational databases.
- It "has become the de facto standard in the relational database world" (Chris Date).
- SQL is English-like and non-procedural and has a small number of statement types, yet is flexible and powerful.
- SQL includes facilities for creating, retrieving and updating tables and for controlling access to the database.
- SQL stands for Structured Query Language.
- SQL processes data at the set level, which means it works with groups of items at a time, e.g., entire tables.

Standards:
- There is an ANSI-approved standard for SQL.
- First standardised in 1986 (ANSI X3.135)Latest update version was in 2016. Now covers 9 parts, such as the foundations and persistent stored modules, management of external data, object language bindings
- See this for further details: https://blog.ansi.org/2018/10/sql-standard-iso-iec-9075-2016-ansi-x3-135/
- See what is new in the 2023 standard: http://peter.eisentraut.org/blog/2023/04/04/sql-2023-is-finished-here-is-whats-new


## Relational Databases

Relational databases follow a set of rules and regulations, such as:

**Columns**
- these are the attributes of a relations
- the heading is the name of the column and is unique
- there is a fixed number of columns
- the order is irrelevant
- takes values from a set of legal values (data integrity)
- number of columns = degree of a relation

**Rows**
- called tuples
- variable number of rows
- all rows have the same structure
- number of rows = cardinality

**Keys**
- each relation should have a way to uniquely identify each row: primary key
- a key is the minimal set of attributes to uniquely identify each tuple

Different types of keys:
- *Candidate key:* some relations can have more than one potential key
- *Primary key:* the candidate key selected to be the primary key
- *Composite key:* set of attributes that can serve as a key
- *Surrogate key:* automatically generated key

**Relationships**
- established by foreign keys
- a foreign key is usually the primary key from the table that it is related to, e.g., a relation Employee could have a foreign key `department_no`, which is the primary key of the Department relation.
- the values in a foreign key must match the values in the related primary key
- if the relationship is mandatory, then the foreign key field can not contain nulls
- if the relationship is optional, then the foreign key can contain nulls

**Integrity constraints**
- Entity integrity: no part of a primary key can be null
- Referential integrity: foreign keys must either be null, or contain a value from the primary key it is related to

### Structured data

Relational databases work best with structured data, where the data has the same:
- number of columns per row
- data types in each column

For example, the patient table seen in the `Part 08 Notebooks:`

![Patient table](images/patient.jpg)


### Semi-structured data

Semi-structured data can differ from this in that each row or document can vary in size.

Rows are called documents in a `Document databases`, such as `MongoDB`. Data is stored in JSON format and each document can have a different structure, where documents can have sub-documents and each key/value pair can differ in size .

For example, the followings take the first and last row from a JSON dataset generated from Twitter (now X) on 11th January 2023 (`#BBCNews`):

In [ ]:
! head -1 data/BBCNews-230111-2214.json

In [ ]:
! tail -1 data/BBCNews-230111-2214.json

It is hard to read the data in this format, but from a quick glance we can see the `public_metric` key has a different amount of items in each value:

<pre>
"public_metrics": {"reply_count": 2, "like_count": 3, "impression_count": 486}
"public_metrics": {"retweet_count": 17, "reply_count": 23, "like_count": 69, "quote_count": 2, "impression_count": 123806}
</pre>

This type of data is better suited to document databases, which will be looked at in Weeks 14 and 15 and later Tutorials. Most relational databases, however, offer support for JSON data, for example: https://www.postgresql.org/docs/current/functions-json.html

## Notebook Dataset

This Notebook uses the *Cellphone Recommendations* example from Kaggle.com

https://www.kaggle.com/datasets/meirnizri/cellphones-recommendations

About Dataset - from the above website (Accessed 16/11/2022):

*This dataset contains three files:*

*The cellphone data.csv contains data on the most popular cell phones in the US in 2022. The data for each cell phone consists of the most notable features such as performance rating (AnTuTu), memory size, camera's resolution, battery size, screen size, release date, etc. The price of each cell phone collected from Amazon and Best-Buy (in Aug 22). Overall, in our dataset there are 34 cell phones with 13 features.*

*The user's data and their ratings are in cellphones users.csv and cellphones ratings.csv. To elicit the ratings, we conducted a survey on Mechanical Turk. Each participant was presented with 10 random cell phones, and she was asked to indicate how likely she is to purchase each of the cell phones at the given price, on a scale from 1 (very unlikely) to 10 (very likely). We also asked each participant to add personal information: age, gender, and occupation.*

*This dataset can be used for building a recommendation system model that relies mainly on the features of the items.*

License: Database: Open Data Commons Open Database License (https://opendatacommons.org/licenses/odbl/1-0/)

### Accessing PostgreSQL

Enable access to the PostgreSQL database engine via SQL cell magic.

In [ ]:
%load_ext sql

Use the sql_init.ipynb file provided by Tutorial 08.2 to login (User tm351)

In [ ]:
# Make the connection - this file is available from the Notebooks 08 folder

%run sql_init.ipynb

If this has run properly it will have setup a connection string (DB_CONNECTION_STRING), which can be used to create a connection to the database (DB_CONNECTION).

The connection string is made up of several parts:

- `postgresql` : states that we will use PostgreSQL as our database engine
- The user ID and password being used to make the connection, separated by a colon (the user ID and password are both tm351)
- `localhost:5432` : the port on which the database engine is listening
- Finally, the string contains the name of the database (`tm351`)

See the Notebooks in Part 08 for further examples.

In [ ]:
DB_CONNECTION_STRING

In [ ]:
import sqlalchemy 
DB_CONNECTION=sqlalchemy.create_engine(DB_CONNECTION_STRING)
DB_CONNECTION

In [ ]:
%sql DB_CONNECTION

### Setting up the Sample Database

Drop the tables if they previously existed - only run this if they exist:

In [ ]:
%%sql
/* need to drop cellphone_ratings first, since it is related to the other two tables */
DROP TABLE IF EXISTS cellphone_ratings;
DROP TABLE IF EXISTS cellphone_data;
DROP TABLE IF EXISTS cellphone_users;


The cellphones dataset is stored in three CSV files. 

*Notebook 08.3 Adding column constraints* to tables shows how we can load these into a Panda DataFrame, then convert to SQL Table. Reading CSV files into a Pandas dataframe is first shown in *Notebook 02.2.1 Data file formats - CSV*

Import the CSV files into Panda dataframes:

In [ ]:
# Import the cellphones_data.csv file into a DataFrame and change cellphones to cellphone
cellphone_data_df=pd.read_csv('./data/cellphones data.csv',
                       parse_dates=['release date'], dayfirst=True)

#Look at the first few rows of the resulting DataFrame
cellphone_data_df.head()

In [ ]:
# change spaces to underscore (_) in column names, makes life easier when querying the tables

cellphone_data_df = cellphone_data_df.rename(columns={'operating system': 'operating_system', 'internal memory': 'internal_memory',
                                                       'main camera' : 'main_camera', 'selfie camera' : 'selfie_camera',
                                                       'battery size' : 'battery_size','screen size' : 'screen_size',
                                                       'release date' : 'release_date'})

cellphone_data_df.head()

In [ ]:
# Import the cellphones ratings.csv file into a DataFrame
cellphone_ratings_df=pd.read_csv('./data/cellphones ratings.csv')

#Look at the first few rows of the resulting DataFrame
cellphone_ratings_df.head()

In [ ]:
# Import the cellphones users.csv file into a DataFrame
cellphone_users_df=pd.read_csv('./data/cellphones users.csv')

#Look at the first few rows of the resulting DataFrame
cellphone_users_df.head()

Now convert to tables

Postgresql allows the data to be imported via a Panda’s DataFrame

See Notebook 08.2 for some examples


In [ ]:
cellphone_data_df.to_sql('cellphone_data',
                  DB_CONNECTION,
                  if_exists='replace',
                  index=False
                  )

In [ ]:
cellphone_ratings_df.to_sql('cellphone_ratings',
                  DB_CONNECTION,
                  if_exists='replace',
                  index=False
                  )

In [ ]:
cellphone_users_df.to_sql('cellphone_users',
                  DB_CONNECTION,
                  if_exists='replace',
                  index=False
                  )

Check tables have been created ok.

In [ ]:
%%sql
SELECT * FROM cellphone_users;

In [ ]:
%%sql
SELECT * FROM cellphone_ratings;

In [ ]:
%%sql
SELECT * FROM cellphone_data;

When there are a lot of rows LIMIT can be used to restrict how many rows are returned - similar to the .head() function

In [ ]:
%%sql

SELECT *
FROM cellphone_data
LIMIT 5;

Can use OFFSET to skip n records before applying the LIMIT

In [ ]:
%%sql

SELECT *
FROM cellphone_data
LIMIT 5 OFFSET 5;

LIMIT does not take negative arguments, for example, to get last 5 records, but you can sort the records in ascending/descending order to achieve the same effect:

In [ ]:
%%sql
SELECT * FROM cellphone_ratings
ORDER by rating, cellphone_id
LIMIT 10;

In [ ]:
%%sql
/*  
    Note if we want both columns in descending order, you need to use DESC twice, otherwise it will default to ascending
*/
SELECT * FROM cellphone_ratings
ORDER by rating DESC, cellphone_id DESC
LIMIT 10;

In [ ]:
%%sql
/*  
    Note if we want both columns in descending order, you need to use DESC twice, otherwise it will default to ascending
*/
SELECT * FROM cellphone_ratings
ORDER by user_id desc
LIMIT 10;

You can use the data dictionary tables to check that the tables have been created:

In [ ]:
%%sql
/* check if tables created */

SELECT *
FROM information_schema.tables
WHERE table_type = 'BASE TABLE' and table_schema <> 'pg_catalog'
and table_name LIKE 'cellphone%';

SQL *INSERT* command can be also used to add some data.

We can check the structure of the tables first

In [ ]:
%reload_ext schemadisplay_magic

In [ ]:
%schema --connection_string $DB_CONNECTION_STRING -t cellphone_data

In [ ]:
%schema --connection_string $DB_CONNECTION_STRING -t cellphone_ratings

In [ ]:
%schema --connection_string $DB_CONNECTION_STRING -t cellphone_users

Before making any changes to the data, lets add some constraints, such as a PRIMARY KEY for each table:


In [ ]:
%%sql

ALTER TABLE cellphone_data
ADD CONSTRAINT cellphone_data_pk
    PRIMARY KEY(cellphone_id);

In [ ]:
%%sql

ALTER TABLE cellphone_users
ADD CONSTRAINT cellphone_users_pk
    PRIMARY KEY(user_id);

cellphones_ratings needs a composite key, since a user may have rated more than one cellphone.

In [ ]:
%%sql

ALTER TABLE cellphone_ratings
ADD CONSTRAINT cellphone_ratings_pk
    PRIMARY KEY(user_id, cellphone_id);

Plus add some foreign keys to cellphone_ratings and cellphone_users

In [ ]:
%%sql

ALTER TABLE cellphone_ratings
ADD CONSTRAINT cellphone_users_fk
    FOREIGN KEY(user_id) REFERENCES cellphone_users(user_id);

In [ ]:
%%sql

ALTER TABLE cellphone_ratings
ADD CONSTRAINT cellphone_data_fk
    FOREIGN KEY(cellphone_id) REFERENCES cellphone_data(cellphone_id);

In [ ]:
%schema --connection_string $DB_CONNECTION_STRING -t cellphone_data

In [ ]:
%%sql
/* add a user and their rating for a phone */
INSERT INTO cellphone_users VALUES (300, 61, 'Female', 'OU Associate Lecturer');
INSERT INTO cellphone_ratings VALUES (300, 8, 9);
/* add a user who has not made a rating and a phone that has not been rated yet */
INSERT INTO cellphone_users VALUES (350, 40, 'Male', 'Contract Administrator');
INSERT INTO cellphone_data VALUES (40, 'OPPO', 'A79', 'Android', 128, 8, 5.76, 50, 2, 5000, 6.72, 218, 154.99, '2023-10-28');
COMMIT;

Phone spec taken from: https://specs-tech.com/en/oppo-a79/ and price Amazon UK 14/11/23

Test that the primary key and foreign keys work

The following three inserts should generate an integrity error - can you see why?


In [ ]:
%%sql
INSERT INTO cellphone_users VALUES (300, 21, 'Male', 'IT Consultant');

In [ ]:
%%sql
INSERT INTO cellphone_ratings VALUES (400, 9, 9);

In [ ]:
%%sql
INSERT INTO cellphone_ratings VALUES (300, 50, 9);

## Querying the data
### Basics: Single Table Queries

The basic SELECT statement is:

<pre>
SELECT [DISTINCT] { * | column_list}
    FROM tablename [, tablename ] 
    [WHERE condition]
    [GROUP BY column_list]
    [HAVING condition]
    [ORDER BY column_list]
	[LIMIT n]
</pre>

Everything in square brackets [] is optional

For example:

SELECT DISTINCT city FROM Student
WHERE REGION = ‘R04’;


Some queries
=======

Simple selects


In [ ]:
%%sql
SELECT * FROM cellphone_data;

In [ ]:
%%sql
/* restrict columns */

SELECT model, cellphone_id, brand  FROM cellphone_data;

In [ ]:
%%sql
/* restrict rows */

SELECT * FROM cellphone_data WHERE brand = 'Samsung';

In [ ]:
%%sql
/* combination */

SELECT cellphone_id, brand, model FROM cellphone_data
WHERE brand = 'Apple';

Joins
===

In [ ]:
%%sql
/* traditional way to join tables, using table aliases */

SELECT cr.cellphone_id, brand, model, rating 
FROM cellphone_data cd, cellphone_ratings cr
WHERE cd.cellphone_id = cr.cellphone_id;

Hint: do check the number of rows returned from any join, you should never get more rows than there are in any of the tables used. If you do, check that the tables are joined correctly.

In [ ]:
%%sql
/* traditional way to join, restricting to Samsung only */

SELECT cr.cellphone_id, brand, model, rating 
FROM cellphone_data cd, cellphone_ratings cr
WHERE cd.cellphone_id = cr.cellphone_id
AND brand = 'Samsung';


In [ ]:
%%sql
/* ANSI join */

SELECT cr.cellphone_id, brand, model, rating 
FROM cellphone_data cd JOIN cellphone_ratings cr 
ON cd.cellphone_id = cr.cellphone_id
AND brand = 'Samsung';


In [ ]:
%%sql
/* ANSI join */

SELECT cu.user_id, age, occupation, cr.user_id, cellphone_id, rating 
FROM cellphone_users cu JOIN cellphone_ratings cr 
ON cu.user_id = cr.user_id;

In [ ]:
%%sql
/* Join all three tables */ 
SELECT cu.user_id, age, occupation, cr.user_id, cr.cellphone_id, rating 
FROM cellphone_users cu, cellphone_ratings cr, cellphone_data cd
WHERE cr.cellphone_id = cd.cellphone_id
    AND cu.user_id = cr.user_id;

In [ ]:
%%sql
/* Join all three tables - ANSI syntax */ 
SELECT cu.user_id, age, occupation, cr.user_id, cr.cellphone_id, rating 
FROM 
    cellphone_users cu 
JOIN cellphone_ratings cr ON cu.user_id = cr.user_id
JOIN cellphone_data cd ON cr.cellphone_id = cd.cellphone_id;

JOINs work by matching values in the columns used for the join. If you have an optional relationship then some rows may not appear in the results. An `OUTER JOIN` can be used to include these rows too.

First, just check how many rows are in each table:

In [ ]:
%%sql
SELECT COUNT(*) AS data_count FROM cellphone_data;

In [ ]:
%%sql
SELECT COUNT(*) AS ratings_count FROM cellphone_ratings;

In [ ]:
%%sql
SELECT COUNT(*) AS ratings_count FROM cellphone_users;

In [ ]:
%%sql
/*
    outer join to include cellphones with no ratings
*/

SELECT cellphone_data.cellphone_id, brand, model, cellphone_ratings.cellphone_id, rating 
FROM cellphone_data LEFT OUTER JOIN cellphone_ratings 
ON cellphone_data.cellphone_id = cellphone_ratings.cellphone_id;

In [ ]:
%%sql
/* outer join to include cellphones with no ratings only */

SELECT cellphone_data.cellphone_id, brand, model, cellphone_ratings.cellphone_id, rating 
FROM cellphone_data LEFT OUTER JOIN cellphone_ratings 
ON cellphone_data.cellphone_id = cellphone_ratings.cellphone_id
AND cellphone_ratings.cellphone_id IS NULL;


Our newly added OPPO phone is not the only one without a rating

In [ ]:
%%sql
/*  outer join
    to include users who have not rated any phones
    our added user is at the end
*/

SELECT cu.user_id, age, gender, rating
FROM cellphone_ratings cr RIGHT OUTER JOIN cellphone_users cu 
ON cr.user_id = cu.user_id ;

In [ ]:
%%sql
/*  outer join
    to only show users who have not rated any phones
*/

SELECT cu.user_id, age, gender, rating
FROM cellphone_ratings cr RIGHT OUTER JOIN cellphone_users cu ON cr.user_id = cu.user_id 
AND cu.user_id IS NULL
ORDER BY user_id;

The other type of OUTER JOIN is a *FULL OUTER JOIN* which would be useful if we had some rows in the cellphone_ratings table that did not match either the ratings or users tables, but the foreign key constraints would prevent this.

In [ ]:
%%sql
/*  full outer join
    to only show users who have not rated any phones, or cellphone ratings without a user. 
    In this case the foreign key constraint will mean the results are similar to the RIGHT OUTER JOIN above, without 
    the test for nulls
*/

SELECT cu.user_id, age, gender, rating
FROM cellphone_ratings cr FULL OUTER JOIN cellphone_users cu ON cr.user_id = cu.user_id 
ORDER BY user_id;

In [ ]:
%%sql
/*
    cartesian product
    look what happens if you forget to join the tables.
*/

SELECT *
FROM cellphone_ratings, cellphone_users;


Always check how many rows are returned. You should not get back more rows than there are in the tables!

In [ ]:
%%sql
/* statistics */

SELECT cellphone_id, count(*) as rating_count
FROM cellphone_ratings
GROUP BY cellphone_id
ORDER BY cellphone_id;

Would the following query be correct:

In [ ]:
%%sql
SELECT cellphone_id, count(*) as rating_count
FROM cellphone_ratings
WHERE count(*) > 35
GROUP BY cellphone_id
ORDER BY cellphone_id;

No, the above query should not work, because the WHERE clause is referencing a GROUP function. If you want to restrict the rows used in the group function, use HAVING instead:

In [ ]:
%%sql
/* Having is like a WHERE clause on the group function: */

SELECT cellphone_id, count(*) as rating_count
FROM cellphone_ratings
GROUP BY cellphone_id
HAVING count(*) > 25
ORDER BY cellphone_id;


In [ ]:
%%sql
/* Can have a WHERE and HAVING clause together */

SELECT cellphone_id, count(*) as rating_count
FROM cellphone_ratings
WHERE rating > 6
GROUP BY cellphone_id
HAVING count(*) > 25
ORDER BY cellphone_id;

Subqueries:

In [ ]:
%%sql
/* Who is the eldest user? */

SELECT user_id, age, gender FROM cellphone_users WHERE age = 
    (SELECT MAX(age) FROM cellphone_users);

In [ ]:
%%sql
/*
    Which users work in IT.
    Note, you need to use IN rather than equals (=) for comparison, 
    since more than one user_id will be returned.
*/

SELECT user_id, age, gender
FROM cellphone_users 
WHERE user_id IN 
    (SELECT user_id FROM cellphone_users WHERE occupation = 'IT');

In [ ]:
%%sql
/* 
    Which phone has the highest rating:
*/

SELECT cellphone_data.cellphone_id, brand, model, rating 
FROM cellphone_data JOIN cellphone_ratings 
ON cellphone_data.cellphone_id = cellphone_ratings.cellphone_id
WHERE rating = (SELECT MAX(rating) FROM cellphone_ratings);

Perhaps an anomaly, since the other ratings seem to be 1-10

In [ ]:
%%sql
/*
    Column subquery:
*/

SELECT cellphone_id, brand, model, 
    (SELECT MAX(price) as Max_Price FROM cellphone_data), 
    (SELECT MAX(price) FROM cellphone_data)-price as Difference 
FROM cellphone_data;


Correlated subquery

This will find the employees whose salary is 10% more than the average salaries for employees of the same gender.

This requires comparing a person’s salary with an aggregate (average of all salaries):


In [ ]:
%%sql
/*
    correlated subquery
    which phones cost more than average price
    of phones of the same brand
*/

SELECT brand, model, price FROM cellphone_data cd1 
WHERE price > 
    (SELECT AVG(price) AS Avg_price 
     FROM cellphone_data cd2 
     WHERE cd1.brand = cd2.brand);


## Exercises: Name that Query

The following queries are based on the cellphones schema. What do you think they mean? 

**Query 1**
<pre>
SELECT * FROM cellphone_users;
</pre>

**Query 2**
<pre>
SELECT brand, model, release_date 
FROM cellphone_data
WHERE release_date > '2022-01-01';
</pre>

Will the next two queries return the same results?

**Query 3**
<pre>
SELECT brand, price 
FROM cellphone_data
WHERE (brand = 'Samsung' OR brand = 'OnePlus') 
AND price >= 600;
</pre>

**Query 4**
<pre>
SELECT brand, price 
FROM cellphone_data
WHERE brand = 'Samsung' OR (brand = 'OnePlus' 
AND price >= 600);
</pre>                                                                   

**Query 5**
<pre>
SELECT brand, ROUND(AVG(price),2) AS average_price FROM cellphone_data 
GROUP BY brand
ORDER BY average_price DESC;</pre>

**Query 6**
<pre>
SELECT AVG(price) FROM cellphone_data GROUP BY brand;</pre>

### Exercises: Name that error

Why do these generate errors:

**Query 7**
<pre>
SELECT brand, model, rating 
FROM cellphone_data, cellphone_ratings 
WHERE cellphone_id = cellphone_id
    AND rating < 3;</pre>

**Query 8**
<pre>
SELECT cellphone_id, brand, model, rating 
FROM cellphone_data cd, cellphone_ratings cr
WHERE cd.cellphone_id = cr.cellphone_id
    AND rating > 7
ORDER BY rating desc;

**Query 9**
<pre>
SELECT model, AVG(price) FROM cellphone_data GROUP BY operating_system;</pre>


### Exercises: Subqueries

Do the following two queries return the latest or earliest releases:

**Query 10**
<pre>
/* latest or earliest release? */
SELECT * FROM cellphone_data WHERE release_date =
(SELECT MIN(release_date) FROM cellphone_data);</pre>


**Query 10**
<pre>
SELECT * FROM cellphone_data WHERE release_date =
(SELECT MAX(release_date) FROM cellphone_data);</pre>

Why use `IN` instead of equals (=):

**Query 12**
<pre>
SELECT * FROM cellphone_data WHERE brand IN
(SELECT brand FROM cellphone_data cd, cellphone_ratings cr
WHERE cd.cellphone_id = cr.cellphone_id 
    AND rating > 8);</pre>

### Final Exercises

The following exercises use a smaller DEPT/EMP dataset:

![Dept table](images/dept.png)

![Employee table](images/emp.png)

Which of the following SQL commands answer the question given:

List the empno, name and salary of employees in department 10

<pre>
1. SELECT * FROM Emp WHERE deptno = 10;
2. SELECT empno, ename, sal FROM Emp;
3. SELECT empno, ename, sal FROM Emp WHERE deptno = 10;
4. SELECT empno, name, salary FROM Emp WHERE deptno = 10;
</pre>

Find the youngest employee:

<pre>
1. SELECT * FROM Emp WHERE dob = ‘1992-11-01’;
2. SELECT * FROM Emp WHERE empno = 4444;
3. SELECT * FROM Emp WHERE dob =
	(SELECT MAX(dob) FROM EMP);
4. SELECT * FROM Emp WHERE dob =
	(SELECT MIN(dob) FROM EMP);
</pre>

List the department number, name and employee names for everyone assigned a department:

<pre>
1. SELECT * FROM Dept, Emp;
2. SELECT deptno, dname, ename FROM Dept, Emp;
3. SELECT deptno, dname, ename FROM Dept, Emp WHERE dept.deptno = emp.deptno;
4. SELECT dept.deptno, dname, ename FROM Dept, Emp WHERE dept.deptno = emp.deptno;
</pre>

List the average salary by department:

<pre>
1. SELECT deptno, AVG(sal) FROM Emp;
2. SELECT deptno, AVG(sal) FROM Emp GROUP BY deptno;
3. SELECT deptno, AVG(sal) FROM Emp GROUP BY sal;
4. SELECT AVG(sal) FROM Emp GROUP BY deptno;
</pre>

## Useful Notebooks

Using pandasql library:
- 03.2 Selecting and projecting, sorting and limiting

Using PostgreSQL:
- 03.3 Combining data from multiple datasets
- 03.4 Handling missing data
- 04.5 Split-apply-combine -with SQL and pandas
- Part 08 – 12 notebooks. 
Uses hospital example


## Answers

### Name that Query

**Query 1**

In [ ]:
%%sql
/* retrieve all rows from cellphone_users */

SELECT * FROM cellphone_users;

**Query 2**

In [ ]:
%%sql
/*  retrieve cellphones released after the 1st day of 2022 */

SELECT brand, model, release_date 
FROM cellphone_data
WHERE release_date > '2022-01-01';

**Query 3**

In [ ]:
%%sql
/*  retrieve brand and price of all Samsung and OnePlus phones over £600 */

SELECT brand, price 
FROM cellphone_data
WHERE (brand = 'Samsung' OR brand = 'OnePlus') 
AND price >= 600;

**Query 4**

In [ ]:
%%sql
/*  retrieve brand and price of all Samsungs or OnePlus phones over £600.
    Notice the subtle difference from previous query */

SELECT brand, price 
FROM cellphone_data
WHERE brand = 'Samsung' OR (brand = 'OnePlus' 
AND price >= 600);

**Query 5**

In [ ]:
%%sql
SELECT brand, ROUND(AVG(price),2) AS average_price FROM cellphone_data 
GROUP BY brand
ORDER BY average_price DESC;

**Query 6**

In [ ]:
%%sql
SELECT AVG(price) FROM cellphone_data GROUP BY brand;

### Exercises: Why the error?

**Query 7**

In [ ]:
%%sql
SELECT brand, model, rating 
FROM cellphone_data, cellphone_ratings 
WHERE cellphone_id = cellphone_id
AND rating < 3;

We can see from the error message that it does not like the last line, this is because `cellphone_id` appears in both tables. Including the table name is optional when referring to a column, but if two or more tables in a query have a column with the same name, then you must include either the table name or a table alias to distinguish between the columns with the same names.

For example:

In [ ]:
%%sql
SELECT brand, model, rating 
FROM cellphone_data, cellphone_ratings 
WHERE cellphone_data.cellphone_id = cellphone_ratings.cellphone_id
    AND rating < 3;

Or use a table alias. 

In [ ]:
%%sql
SELECT brand, model, rating 
FROM cellphone_data cd, cellphone_ratings cr 
WHERE cd.cellphone_id = cr.cellphone_id
    AND rating < 3;

Note, if you do include aliases, then they must be used throughout the query when referring to a column, otherwise you will get a different error message:

In [ ]:
%%sql
SELECT brand, model, rating 
FROM cellphone_data cd, cellphone_ratings cr 
WHERE cellphone_data.cellphone_id = cr.cellphone_id
    AND rating < 3;

**Query 8**

In [ ]:
%%sql
SELECT cellphone_id, brand, model, rating 
FROM cellphone_data cd, cellphone_ratings cr
WHERE cd.cellphone_id = cr.cellphone_id
    AND rating > 7
ORDER BY rating desc;

This is similar to above - cellphone_id appears in the SELECT statement and also needs to be distinguished, in this case using the table alias.

In [ ]:
%%sql
SELECT cd.cellphone_id, brand, model, rating 
FROM cellphone_data cd, cellphone_ratings cr
WHERE cd.cellphone_id = cr.cellphone_id
    AND rating > 7
ORDER BY rating desc;

Hmm, looks like there is an outlier there with the 18 - assuming the rating should be between 1 and 10!

**Query 9**

In [ ]:
%%sql
SELECT model, AVG(price) FROM cellphone_data GROUP BY operating_system;

When mixing columns and aggregate functions, as seen in this query, the column seen in the SELECT statement needs to be used in the `GROUP BY` command. In this case we want to average the prices by operating system and let's round the prices to 2 decimals:

In [ ]:
%%sql
SELECT operating_system, ROUND(AVG(price),2) AS price_avg FROM cellphone_data GROUP BY operating_system;

**Query 10**

In [ ]:
%%sql
SELECT * FROM cellphone_data WHERE release_date =
(SELECT MIN(release_date) FROM cellphone_data);

**Query 11**

In [ ]:
%%sql
SELECT * FROM cellphone_data WHERE release_date =
(SELECT MAX(release_date) FROM cellphone_data);

For dates MIN() returns the earliest date and MAX() the latest.

**Query 12**

In [ ]:
%%sql
SELECT * FROM cellphone_data WHERE brand IN
(SELECT brand FROM cellphone_data cd, cellphone_ratings cr
WHERE cd.cellphone_id = cr.cellphone_id 
    AND rating > 8);

Note, if the subquery is likely to return more than one value, then you must use `IN` instead of `=` for the comparison.

### Final queries

These queries are based on a simple DEPT and EMP scenario (where 1 department can employ many employees, each employee may work for one department):

In [ ]:
%%sql

/* Dept and Emp Schema */

CREATE TABLE Dept (
    deptno NUMERIC(2) PRIMARY KEY,
    dname VARCHAR(20)
);

CREATE TABLE Emp (
    empNo CHAR(6) PRIMARY KEY,
    eName VARCHAR(20), 
    dob DATE, 
    sal DECIMAL(8,2), 
    gender CHAR(1) CHECK (gender IN ('M','F')),
    deptno NUMERIC(2) REFERENCES Dept
);

In [ ]:
%%sql
/* insert some records */
INSERT INTO Dept VALUES (10,'Finance');
INSERT INTO Dept VALUES (20,'Sales');
INSERT INTO Dept VALUES (30,'Marketing');

INSERT INTO Emp VALUES ('1111','June','1973/08/18',35000,'F', 10);
INSERT INTO Emp VALUES ('2222','Fred','1981/04/22',50500,'M', 20);
INSERT INTO Emp VALUES ('3333','Tom','1985/07/11',21570,'M', 20);
INSERT INTO Emp VALUES ('4444','Judith','1992/11/01',44500,'F',NULL);

COMMIT;

Which of the following SQL commands answer the question given:

List the empno, name and salary of employees in department 10

<pre>
1. SELECT * FROM Emp WHERE deptno = 10; /* correct department, but we only wanted 3 columns */
2. SELECT empno, ename, sal FROM Emp; /* correct columns, but will return all rows */
3. SELECT empno, ename, sal FROM Emp WHERE deptno = 10; /* correct - right columns and rows */
4. SELECT empno, name, salary FROM Emp WHERE deptno = 10; /* correct rows, but the salary column is just called sal */
</pre>

In [ ]:
%%sql 
SELECT empno, ename, sal FROM Emp WHERE deptno = 10;

Find the youngest employee:

<pre>
1. SELECT * FROM Emp WHERE dob = ‘1992-11-01’; /* this would generate the correct row based on the current data */
2. SELECT * FROM Emp WHERE empno = 4444; /* same as above, but what happens if a new, younger, employee joins the company */
3. SELECT * FROM Emp WHERE dob =
	(SELECT MAX(dob) FROM EMP); /* correct */
4. SELECT * FROM Emp WHERE dob =
	(SELECT MIN(dob) FROM EMP); 
    /* this would be the oldest - perhaps counterintuitative if you think max is the bigger number */
</pre>

In [ ]:
%%sql
SELECT * FROM Emp WHERE dob =
	(SELECT MAX(dob) FROM EMP);

List the department number, name and employee names for everyone assigned a department:
<pre>
1. SELECT * FROM Dept, Emp; /* JOIN missing */
2. SELECT deptno, dname, ename FROM Dept, Emp; /* JOIN missing */
3. SELECT deptno, dname, ename FROM Dept, Emp WHERE dept.deptno = emp.deptno; 
    /* join condition added, but what about that first deptno? */
4. SELECT dept.deptno, dname, ename FROM Dept, Emp WHERE dept.deptno = emp.deptno; /* correct */
</pre>

In [ ]:
%%sql
SELECT dept.deptno, dname, ename FROM Dept, Emp WHERE dept.deptno = emp.deptno;

List the average salary by department:

<pre>
1. SELECT deptno, AVG(sal) FROM Emp; /* need to use GROUP BY if mixing columns and group functions */
2. SELECT deptno, AVG(sal) FROM Emp GROUP BY deptno; /* correct */
3. SELECT deptno, AVG(sal) FROM Emp GROUP BY sal; /* need to GROUP BY the column in the SELECT statment */
4. SELECT AVG(sal) FROM Emp GROUP BY deptno; 
    /* would generate the same average result as query 2, but is meaningless without the deptno */
</pre>

In [ ]:
%%sql
SELECT deptno, AVG(sal) FROM Emp GROUP BY deptno;